In [1]:
import torch
print(torch.cuda.is_available())  # 应该返回 True


True


In [3]:
from transformers import pipeline

print(pipeline('sentiment-analysis')('I love using transformers!'))


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
c:\Users\whz12\anaconda3\envs\LLM\lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[{'label': 'POSITIVE', 'score': 0.9994327425956726}]


In [1]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
print(config)


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

LlamaConfig {
  "_name_or_path": "meta-llama/Llama-3.2-3B-Instruct",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 24,
  "num_hidden_layers": 28,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 32.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.49.0.dev0",
  "use_cache": true,
  "vocab_size": 128256
}



c:\Users\whz12\anaconda3\envs\LLM\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in F:\huggingface_cache\hub\models--meta-llama--Llama-3.2-3B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

import torch
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

quantization_config = BitsAndBytesConfig(
    load_in_8bit=False  # 8-bit 量化（可改为 load_in_4bit=True 进一步降低显存）
)
# 加载tokenizer和模型
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct", cache_dir="F:/transformers_cache")
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-3B-Instruct", quantization_config=quantization_config,
    cache_dir="F:/transformers_cache", 
    device_map={"": 0}  # 强制加载到 GPU 0
)
for name, param in model.named_parameters():
    print(f"{name}: {param.shape}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

model.embed_tokens.weight: torch.Size([128256, 3072])
model.layers.0.self_attn.q_proj.weight: torch.Size([4718592, 1])
model.layers.0.self_attn.k_proj.weight: torch.Size([1572864, 1])
model.layers.0.self_attn.v_proj.weight: torch.Size([1572864, 1])
model.layers.0.self_attn.o_proj.weight: torch.Size([4718592, 1])
model.layers.0.mlp.gate_proj.weight: torch.Size([12582912, 1])
model.layers.0.mlp.up_proj.weight: torch.Size([12582912, 1])
model.layers.0.mlp.down_proj.weight: torch.Size([12582912, 1])
model.layers.0.input_layernorm.weight: torch.Size([3072])
model.layers.0.post_attention_layernorm.weight: torch.Size([3072])
model.layers.1.self_attn.q_proj.weight: torch.Size([4718592, 1])
model.layers.1.self_attn.k_proj.weight: torch.Size([1572864, 1])
model.layers.1.self_attn.v_proj.weight: torch.Size([1572864, 1])
model.layers.1.self_attn.o_proj.weight: torch.Size([4718592, 1])
model.layers.1.mlp.gate_proj.weight: torch.Size([12582912, 1])
model.layers.1.mlp.up_proj.weight: torch.Size([1258

In [2]:
print(model.hf_device_map)


{'': 0}
